In [63]:
import os
import re
import shutil
import subprocess
import src.utils as utils
import src.Parser.verilog_parser as verilog_parser
import src.Parser.bench_parser as bench_parser
import src.Parser.conv as conv
from  src.Netlist.AST import AST,module
# import src.Attacks.SATAttack.SATAttack as satattack
import src.Attacks.SATAttack.benchmarks_custom as benchmarks

In [73]:
cir="v2"
mainpath="input_files/Challenge/HeLLOCTF_2023_challenge_LL/oracle_guided"
lockedpath=f"{mainpath}/locked_bench/{cir}_locked.bench"
unlockedpath=f"{mainpath}/locked_bench/{cir}_locked.bench"

In [74]:
import re
import z3
import time
import attacks.satattack.circuit as circuit
import attacks.satattack.sat_model as sat_model
import src.Attacks.SATAttack.benchmarks_custom as benchmarks
import attacks.satattack.dip_finder as dip_finder
import src.Attacks.SATAttack.oracle_runner as oracle_runner

In [118]:
from attacks.satattack.circuit_solver import solve_ckt

class OracleRunner:
  def __init__(self, ckt,bin=False):
    self.ckt = ckt

  def run(self, inputs):
    """Run a set of inputs against the oracle"""
    return solve_ckt(self.ckt, inputs)

  def run_bin(self, primary_inputs,dip,outputs):
    tmp=""
    for i in primary_inputs:
      if(i in dip):
        tmp+=str(int(bool(dip[i])))+" "
      else:
        tmp+="0 "
    
    cmd="input_files/Challenge/HeLLOCTF_2023_challenge_LL/oracle_guided/oracle/v1 "+tmp
    result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return {x:bool(int(result.stdout.split("\n")[-2][i])) for i,x in enumerate(outputs)}

def _find_key(oracle_io_pairs, key_names,nodes, output_names):
    """
    Find a key that satisfies all DIPs found during the SAT attack.
    This key will be in the set of correct keys.

    oracle_io_pairs: array of dip/output pairs in the form of (dip, output)

    returns: key that satisfies all dip constraints
    """

    s = z3.Solver()

    for io_pair in oracle_io_pairs:
        dip = io_pair[0]
        output = io_pair[1]

        constraint_ckt = circuit.Circuit.specify_inputs(dip, nodes, output_names)
        output_constraints = [constraint_ckt.outputs()[name] == output[name] for name in output.keys()]

        s.add(*output_constraints)

    s.check()
    # print("######################################## ERROR #####################################")
    # print(s)
    model = s.model()
    key = sat_model.extract_from_model(model, key_names, completion=True)
    return key


def _check_key(key):
    locked_ckt = circuit.Circuit.specify_inputs(key, nodes, output_names)
    miter = circuit.Circuit.miter(locked_ckt, oracle_ckt)

    s = z3.Solver()
    s.add(miter.outputs()["diff"] == True)

    return s.check() == z3.unsat

In [121]:
cir="v2"
mainpath="input_files/Challenge/HeLLOCTF_2023_challenge_LL/oracle_guided"
lockedpath=f"{mainpath}/locked_bench/{cir}_locked.bench"
unlockedpath=f"{mainpath}/locked_bench/{cir}_locked.bench"

nodes, output_names = benchmarks.read_nodes_b(lockedpath,sort=False)

key_inputs = [node.name for node in nodes.values() if node.type == "Key Input"]
primary_inputs = [node.name for node in nodes.values() if node.type == "Primary Input"]

print(len(key_inputs),len(primary_inputs))

finder = dip_finder.DipFinder(nodes, output_names)
runner = OracleRunner(unlockedpath)

oracle_io_pairs = []
iterations=0
while finder.can_find_dip():
  dip = finder.find_dip()
  oracle_output = runner.run_bin(primary_inputs,dip,output_names)
  # print(dip)
  # print(oracle_output)
  finder.add_constraint(dip, oracle_output)


  oracle_io_pairs.append((dip, oracle_output))
  iterations += 1

key = _find_key(oracle_io_pairs, key_inputs,nodes, output_names)
key


95 277


{'keyinput_88': True,
 'keyinput_59': True,
 'keyinput_46': True,
 'keyinput_51': True,
 'keyinput_61': True,
 'keyinput_18': True,
 'keyinput_66': True,
 'keyinput_87': True,
 'keyinput_15': True,
 'keyinput_50': True,
 'keyinput_65': True,
 'keyinput_35': True,
 'keyinput_10': True,
 'keyinput_21': True,
 'keyinput_77': True,
 'keyinput_44': True,
 'keyinput_83': True,
 'keyinput_70': True,
 'keyinput_81': True,
 'keyinput_76': True,
 'keyinput_30': True,
 'keyinput_12': True,
 'keyinput_16': True,
 'keyinput_82': True,
 'keyinput_27': True,
 'keyinput_23': True,
 'keyinput_36': True,
 'keyinput_1': True,
 'keyinput_92': True,
 'keyinput_52': False,
 'keyinput_39': True,
 'keyinput_55': False,
 'keyinput_41': True,
 'keyinput_79': True,
 'keyinput_74': True,
 'keyinput_78': True,
 'keyinput_31': True,
 'keyinput_69': True,
 'keyinput_45': True,
 'keyinput_62': True,
 'keyinput_47': True,
 'keyinput_48': True,
 'keyinput_57': True,
 'keyinput_58': True,
 'keyinput_49': True,
 'keyinpu

In [142]:

lockedpath="/home/alira/FYP_FINAL/input_files/benchmark_bench/rnd/apex2_enc05.bench"
unlockedpath="/home/alira/FYP_FINAL/input_files/benchmark_bench/original/apex2.bench"

nodes, output_names = benchmarks.read_nodes_b(lockedpath,sort=False)

key_inputs = [node.name for node in nodes.values() if node.type == "Key Input"]
primary_inputs = [node.name for node in nodes.values() if node.type == "Primary Input"]

oracle_ckt = benchmarks.read_ckt(unlockedpath,"b")
finder = dip_finder.DipFinder(nodes, output_names)
runner = OracleRunner(oracle_ckt)


oracle_io_pairs = []
iterations=0
while finder.can_find_dip():
    dip = finder.find_dip()
    oracle_output = runner.run(dip)
    # print(dip)
    # print(oracle_output)
    finder.add_constraint(dip, oracle_output)


    oracle_io_pairs.append((dip, oracle_output))
    iterations += 1

key = _find_key(oracle_io_pairs, key_inputs,nodes, output_names)
key_str="".join([str(int(bool(key[i]))) for i in key])
diff=0
for i,j in zip(key_str,"1001111001011111101100101000001"):
  diff+=int(i)^int(j)
  # if(i!=j):
  #   print(i,j,int(i)^int(j))
100*diff/len("1001111001011111101100101000001")

38.70967741935484

In [146]:
!/home/alira/FYP_FINAL/attacks/SATC/SAT/spramod-host15-logic-encryption-90a8ca47d847/bin/sld /home/alira/FYP_FINAL/input_files/benchmark_bench/rnd/apex2_enc05.bench /home/alira/FYP_FINAL/input_files/benchmark_bench/original/apex2.bench > tmpkey.txt 2>&1

In [148]:

diff=0
for i,j in zip("1001111001011111101100101000001","1001111001011111101100101000001"):
  diff+=int(i)^int(j)
diff

0

In [137]:
# def _check_key(key):
#     """
#     Check that the key returned from the SAT attack is correct. It
#     does this by creating a miter circuit with a locked version
#     and an oracle. If the diff signal returned from the miter circuit
#     cannot be True, then the circuits are equivalent.

#     key: the key returned from the SAT attaack
#     """

#     locked_ckt = circuit.Circuit.specify_inputs(key, nodes, output_names)
#     miter = circuit.Circuit.miter(locked_ckt, oracle_ckt)

#     s = z3.Solver()
#     s.add(miter.outputs()["diff"] == True)

#     return s.check() == z3.unsat


def _key_string(key):
    def x(name):
        tmp=re.findall("[\d]+",name)
        # print("TMP",tmp)
        if (tmp!=[]):
            return int(tmp[0])
            # return tmp
        else:
            return 0
    ordered_names = sorted(key.keys(), key=x,reverse=True)
    # print(key.keys())
    # print(ordered_names)
    # for i in ordered_names[:-1]:
    #     print(i,end=" ")
    
    # print(ordered_names[-1])
    
    key_string = ""

    for name in ordered_names:
        if key[name]:
            key_string += "1"
        else:
            key_string += "0"

    return ordered_names,key_string

NameError: name 'diff' is not defined

In [8]:
# obj=AST(file_path=lockedpath,rw="w",flag="b",top=cir,filename=f"{cir}_locked",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True,keyinput_ref="keyinput")

extract_modules_def took 0.0016849040985107422 seconds to complete its execution.
Verifiying Input Verilog File
Input Verilog File Verified Without Issue
Generating Logic Locking AST File
gates_module_extraction took 0.003293275833129883 seconds to complete its execution.
Found single bit Explict Wire n241 , Storing in v1.io['wires']
Found single bit Explict Wire n55 , Storing in v1.io['wires']
Found single bit Explict Wire n63 , Storing in v1.io['wires']
Found single bit Explict Wire n98 , Storing in v1.io['wires']
Found single bit Explict Wire n42 , Storing in v1.io['wires']
Found single bit Explict Wire n87 , Storing in v1.io['wires']
Found single bit Explict Wire n139 , Storing in v1.io['wires']
Done Generating Logic Locking AST File
Writing LL file
	 Netlist Not Locked, No Update to Peform
Done Writing LL file


./tmp/tmp_verilog_v1.v:275: Warning: Identifier `\n241' is implicitly declared.
./tmp/tmp_verilog_v1.v:278: Warning: Identifier `\n55' is implicitly declared.
./tmp/tmp_verilog_v1.v:282: Warning: Identifier `\n63' is implicitly declared.
./tmp/tmp_verilog_v1.v:290: Warning: Identifier `\n98' is implicitly declared.
./tmp/tmp_verilog_v1.v:326: Warning: Identifier `\n42' is implicitly declared.
./tmp/tmp_verilog_v1.v:328: Warning: Identifier `\n87' is implicitly declared.
./tmp/tmp_verilog_v1.v:357: Warning: Identifier `\n139' is implicitly declared.


In [6]:
# locked = AST(file_path=f"output_files/{cir}_locked.json",rw='r',filename=f"{cir}_locked_test",locked=True)

extract_modules_def took 0.0009570121765136719 seconds to complete its execution.
Reading LL file
	 Loading json file
	 Done Loading json file
	 Loading Top module in AST
	 Done Loading Top module v1 in AST
	 Loading module data in AST
		 Loading module v1 in AST
		 Done Loading module v1 in AST
	 Done Loading module data in AST
	 Loading gate level data in AST
	 Done Loading gate level data in AST
Done Reading LL file


In [9]:
# locked.top_module.io["inputs"]

{'keyinput_9': {'bits': 1},
 'keyinput_8': {'bits': 1},
 'keyinput_7': {'bits': 1},
 'keyinput_6': {'bits': 1},
 'keyinput_5': {'bits': 1},
 'keyinput_4': {'bits': 1},
 'keyinput_37': {'bits': 1},
 'keyinput_36': {'bits': 1},
 'keyinput_35': {'bits': 1},
 'keyinput_34': {'bits': 1},
 'keyinput_33': {'bits': 1},
 'keyinput_32': {'bits': 1},
 'keyinput_31': {'bits': 1},
 'keyinput_30': {'bits': 1},
 'keyinput_3': {'bits': 1},
 'keyinput_29': {'bits': 1},
 'keyinput_28': {'bits': 1},
 'keyinput_27': {'bits': 1},
 'keyinput_26': {'bits': 1},
 'keyinput_25': {'bits': 1},
 'keyinput_24': {'bits': 1},
 'keyinput_23': {'bits': 1},
 'keyinput_22': {'bits': 1},
 'keyinput_21': {'bits': 1},
 'keyinput_20': {'bits': 1},
 'keyinput_2': {'bits': 1},
 'keyinput_19': {'bits': 1},
 'keyinput_18': {'bits': 1},
 'keyinput_17': {'bits': 1},
 'keyinput_16': {'bits': 1},
 'keyinput_15': {'bits': 1},
 'keyinput_14': {'bits': 1},
 'keyinput_13': {'bits': 1},
 'keyinput_12': {'bits': 1},
 'keyinput_11': {'bits

In [6]:
# satp=satattack.SatAttack(locked_file_type="b",
#                          unlocked_file_type="b",
#                          locked_filename=lockedpath,
#                          unlocked_filename=unlockedpath
#                          )